<a href="https://colab.research.google.com/github/sadbinsiddique/cvpr-face-detection/blob/main/face_recognition_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50, MobileNetV2

In [ ]:
# Check if TensorFlow is using GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU is being used")
    except RuntimeError as e:
        print("❌ Error setting GPU memory growth:", e)
else:
    print("❌ No GPU found, using CPU instead")

In [ ]:
# ==== CONFIGURATION ====
dataset_dir = "/content/drive/MyDrive/images (1)"
image_size = 237
batch_size = 16
epochs = 30

# ==== DATA AUGMENTATION ====
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


In [ ]:
# ==== CLASS COUNT & CONFIG ====
num_classes = train_data.num_classes

if num_classes == 2:
    final_activation = 'sigmoid'
    loss_fn = 'binary_crossentropy'
    output_units = 1
else:
    final_activation = 'softmax'
    loss_fn = 'categorical_crossentropy'
    output_units = num_classes

In [ ]:
# ==== SAVE CLASS INDICES ====
with open("class_indices.json", "w") as f:
    json.dump(train_data.class_indices, f)
print("✅ Class indices saved to class_indices.json")

In [ ]:
# ==== BASE MODELS ====
input_tensor = Input(shape=(image_size, image_size, 3))

effnet = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)

for layer in effnet.layers: layer.trainable = False
for layer in resnet.layers: layer.trainable = False
for layer in mobilenet.layers: layer.trainable = False

x1 = GlobalAveragePooling2D()(effnet.output)
x2 = GlobalAveragePooling2D()(resnet.output)
x3 = GlobalAveragePooling2D()(mobilenet.output)

merged = Concatenate()([x1, x2, x3])
x = Dense(512, activation='relu')(merged)
x = Dropout(0.5)(x)
output = Dense(output_units, activation=final_activation)(x)

model = Model(inputs=input_tensor, outputs=output)

In [ ]:
# ==== COMPILE ====
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [ ]:
# ==== SUMMARY & CALLBACKS ====
model.summary()

log_dir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
callbacks = [
    TensorBoard(log_dir=log_dir),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_multimodel.h5", monitor="val_accuracy", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

In [ ]:
# ==== TRAIN ====
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=callbacks
)

In [ ]:
# ==== SAVE ====
model.save('best_model.h5')
print("✅ Final model saved!")

In [ ]:
# ==== EVALUATION ====
val_steps = val_data.samples // val_data.batch_size + 1
val_data.reset()
preds = model.predict(val_data, steps=val_steps)

if num_classes == 2:
    y_pred = (preds > 0.5).astype(int).flatten()
else:
    y_pred = np.argmax(preds, axis=1)

y_true = val_data.classes
labels = list(train_data.class_indices.keys())

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=labels))
print("=== Confusion Matrix ===")
print(confusion_matrix(y_true, y_pred))

In [ ]:
# ==== DASHBOARD ====
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.legend()

plt.subplot(2, 2, 3)
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.subplot(2, 2, 4)
report = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
scores = {m: [report[l][m] for l in labels] for m in ['precision', 'recall', 'f1-score']}

x = np.arange(len(labels))
w = 0.2
plt.bar(x - w, scores['precision'], w, label='Precision')
plt.bar(x, scores['recall'], w, label='Recall')
plt.bar(x + w, scores['f1-score'], w, label='F1-Score')
plt.xticks(ticks=x, labels=labels)
plt.ylim(0, 1.05)
plt.title('Per-Class Metrics')
plt.legend()
plt.tight_layout()

plt.show()
